In [137]:
# Qn 1 (10 points)
# Read the file ‘grades.csv’. Write code that finds and displays the 5 student IDs 
# and their corresponding grades who received the lowest grades in the class. 
# Some students have not appeared for the exam, but they are not to be treated as 0. 
# They need to be simply skipped from the computation. (20points)

import zipfile
import os
import csv

#First unzip the file...
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#We'll open the contents and read them into the buffer

with open("grades.csv", 'r') as csv_file:
    file_contents = csv_file.read()

#print(file_contents)
#If I split by comma now, I end up with a list of items that are combined but no related, and need to be split by \n still.
lines = file_contents.split("\n")
#I can't split the resulting item by ',' because 'list object has no attribute split.' I don't totally understand that yet.
#I can iterate through each element of the new list however and split it that way.

scores_only = []
for item in lines:
    fields = item.split(',')
    #When I print the second item of the fields, it's fine. When I try to append it, I get 'list index out of range.'
    #print(fields[1])
    #I'll try to append it to the output given a condition.
    if(len(item)>0):
        scores_only.append(fields[1])
    #Now we have a good clean list of scores, including blank and the descriptor.

#print(output)
#When I print the 'sorted' output, it sorts the numbers! That's handy, but they aren't in numerical order.
#I'll try to clean the output list more.

# print(sorted(scores_only))
#for item in scores_only:
    # if(type(item) == int): ##This won't work, since nothing in the list is an integer. Instead, we need to see if it's possible to convert to it.
        # score_ints_only.append(item)
#That didn't work, so let's try list comprehension.

#print(scores_only)
# score_ints_only = [item for item in scores_only if isinstance(item, int)]
#That also didn't work because there's a value error for isinstance.
#We're going to resolve this by using a 'try' block.

score_ints_only = []
for item in scores_only:
    try:
        score_ints_only.append(int(item))
    except ValueError:
        continue
#print(sorted(score_ints_only))

#Cool! Now we have a sorted list. We can just print the first few values of this.

lowest_five_values = (sorted(score_ints_only))[:5]

print(f"The lowest five scores, disregarding absent scores, is: {lowest_five_values}")


The lowest five scores, disregarding absent scores, is: [56, 66, 75, 78, 86]


In [161]:
# Qn 2 (20 points)
# Open and read the contents of the file – ‘2_NoofParts_assem.txt’. Perform the following (20 points):
# a)	Calculate how many entries are available in that file excluding the header.
# b)	Calculate the sum of all parts from each file. Essentially, finding the sum of all values contained in the 2nd column of the file.
# c)	Extract the part ID that has the largest associated no. of parts from the entire list.
# '''

file = "2_NoofParts_assem.txt"

import zipfile
import os
import chardet

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#When I try to open here, I get a decode error for utf-8. What type of encoding is this file?

#"rb" does "read binary" so we can look deeply in the file.
with open("2_NoofParts_assem.txt", "rb") as file:
    result = chardet.detect(file.read())
    print(result['encoding'])

with open("2_NoofParts_assem.txt", 'r', encoding = 'utf-16') as file:
    print(file.read())



UTF-16
Part ID	No. of Parts	Parts Completed	Notes
07ca07b4-ba9c-4ee0-b0e7-d9687f23dfe7	159		
0a418501-b2eb-4ce7-86d6-f8050d5a16a4	4		
0a624171-89b8-4cb0-b524-67d351b2712b	410		
0c19e1c1-8c5f-46c4-8bf1-eadd12ae7395	20		
0e8752fa-06d5-46a1-a58f-8a4684d9c728	26		
15e8f97e-5135-4c8a-aa61-07446c4b93c3	293		
189d9799-17b0-416e-979d-acf862d6c98e	437		
19961df6-e2bf-4e9a-9963-d785b451f8f3	51		
1ea29968-cb93-4291-8621-bfc907674443	1		
1ef8ac01-dddd-4b38-88b6-1501b8854030	94		
1f655041-8eec-49f0-8db6-8edee1d3f904	1739		
224bb15e-aed6-4abf-9567-20246dc6e687	9		
2c7f3f59-8729-4325-a139-a38014b87361	34		
31caa0f4-d8c0-48d8-954f-a67bd9510b63	55		
36791e90-896b-4594-b9be-e7fd1f08526a	5		
39204b07-e4c8-4a51-9a57-83c07ef87467	1		
396b4235-c9e5-4cc4-825b-acd7e5522ede	128		
430890c3-d614-4b21-a710-ab392e3ca101	30		
449b315a-bc55-4cb8-9884-fde4513e723c	1		
480e32fb-98a5-4766-b780-fdab9c031d68	19		
4a8c0a68-aaa7-48d1-8377-6a6b41703d8d	700		
4bff6192-fe35-4dda-b7dc-837d917abbf8	571		
57adc8fb-074e-4e3a-9841

In [ ]:
# Qn 3 (25 points)
# Using a python script, open the files contained in the .zip file – “3_Jobs_Completed_log.zip”. 
# Scan the files for the line that starts with the word string – “Jobs Completed.. ”. Extract the number associated with this line and for all instances that this word string appears across all log files, count the total sum across all files contained within the .zip file. (20points) 

# For example:

#  	Jobs Completed.. 10 2018-09-04 08:21:28.503153

# Extract the number 10 from this sentence which signifies the total number of jobs completed at the point in time.  
# Find for all instances in which the string - ‘Jobs Completed’ appears, 
# find the total number of jobs completed by adding the numbers from across the provided log files.

In [ ]:
# Qn 4 (20 points)
# For the same files above, calculate the following:
# a)	How many jobs were completed in the period between 15th Aug 2018 to 15th Sept 2018?
# b)	The day of the year in which the maximum number of jobs was completed.
# c)	Calculate the total number of days elapsed between the time at which the first job batch was completed to the last batch completed.
# '''

In [ ]:
# Qn 5 (25 points)
# Read the following (6_Part1.stl) triangular mesh file available in ASCII format. 
# For those of you aware of 3D printing, you will recognize this is as a .STL file. 
# For those who aren’t aware of what a .STL file is, please read up online on the format of a .STL ascii file. 
# An ASCII formatted .STL file can be opened in any text editor. 
# Every vertex of the triangle is preceeded by the word ‘vertex’ followed by the x, y and z coordinates. 
# Therefore, ONE Triangle should have 3 vertex entries. Typically, the units is not specified, 
# but you can assume it to be in millimeters. 

# Write functions that calculate the following: (20 points)
# a.	Find the total number of triangles listed in the file.
# b.	Store the coordinates of each triangle in a 3-tuple list containing N indices, where N is the total number of triangles.
# c.	List the area of each triangle in the file. Compute the total surface area of all triangles listed in the file. Read up online on the formula to calculate the area of a triangle, given three vertices.
# '''